# FFWIZARD.COM --- QB and DEF Combos

### Import Libraries

In [17]:
import numpy as np
import pandas as pd
import requests
import StringIO
from sklearn import preprocessing
from pandas import ExcelWriter
from datetime import datetime

### Load NFL Schedule

In [18]:
# load nfl schedule data (manually entered)
def load_schedule():
    nfl_sched = pd.read_csv('2017_NFL_Schedule.csv')
    return nfl_sched

### Defense Strength Data

In [19]:
def calc_defense_strength(bye_adjustor):

    # load nfl defense strength data (manually entered)
    def_strength_data = pd.read_csv('Defense_Strength.csv')

    #remove 'source' row
    def_strength = def_strength_data.iloc[:-1, :]

    # make weights based on quality of rankings
    def_rank_weights = pd.Series([0.1, 0.5, 0.3, 0.1])
    
    # create weighted defense rankings array based on weights
    num_teams = len(def_strength)
    weighted_def_rank = np.empty(num_teams, dtype=float)
    
    for i in range(0, num_teams):
        weighted_def_rank[i] = def_rank_weights[0] * float(def_strength.iloc[i, 1]) + def_rank_weights[1] * float(def_strength.iloc[i, 2]) + def_rank_weights[2] * float(def_strength.iloc[i, 3]) + def_rank_weights[3] * float(def_strength.iloc[i, 4])
    weighted_def_rank_df = pd.Series(weighted_def_rank)

    # scale weighted defense rankings to have mean of 0 and std dev of 1
    def_rank_scaled = pd.Series(preprocessing.scale(weighted_def_rank_df))

    # create table with team and score info
    defense_score_no_bye = pd.concat([def_strength.loc[:, "Team"], weighted_def_rank_df, def_rank_scaled], axis=1)
    col_names = ["Team", "Def_Score", "Def_Score_Scaled"]
    defense_score_no_bye.columns = col_names

    # add value for bye weeks in table
    bye_array = pd.DataFrame(["BYE", "XXX", bye_adjustor]).T
    bye_array.columns = col_names
    defense_score = pd.concat([defense_score_no_bye, bye_array])

    defense_score = defense_score.reset_index(drop=True)
    
    return defense_score

### Offense Strength Data

In [20]:
def calc_offense_strength(bye_adjustor):

    # load nfl offense strength data (manually entered)
    off_strength_data = pd.read_csv('Offense_Strength.csv')

    # remove 'source' row
    off_strength = off_strength_data.iloc[:-1, :]

    # make weights based on quality of rankings
    off_rank_weights = pd.Series([0.3, 0.1, 0.3, 0.25, 0.05])
    
    # create weighted offense rankings array based on weights
    num_teams = len(off_strength)
    weighted_off_rank = np.empty(num_teams, dtype=float)
    
    for i in range(0, num_teams):
        weighted_off_rank[i] = off_rank_weights[0] * float(off_strength.iloc[i, 1]) + off_rank_weights[1] * float(off_strength.iloc[i, 2]) + off_rank_weights[2] * float(off_strength.iloc[i, 3]) + off_rank_weights[3] * float(off_strength.iloc[i, 4])  + off_rank_weights[4] * float(off_strength.iloc[i, 5])
    weighted_off_rank_df = pd.Series(weighted_off_rank)

    # scale weighted offense rankings to have mean of 0 and std dev of 1
    off_rank_scaled = pd.Series(preprocessing.scale(weighted_off_rank_df))

    # create table with team and score info
    offense_score_no_bye = pd.concat([off_strength.loc[:, "Team"], weighted_off_rank_df, off_rank_scaled], axis=1)
    col_names = ["Team", "Off_Score", "Off_Score_Scaled"]
    offense_score_no_bye.columns = col_names

    # add value for bye weeks in table
    bye_array = pd.DataFrame(["BYE", "XXX", bye_adjustor]).T
    bye_array.columns = col_names
    offense_score = pd.concat([offense_score_no_bye, bye_array])

    offense_score = offense_score.reset_index(drop=True)
    
    return offense_score

In [21]:
def matchup_scores_by_week(schedule, score, last_week):

    schedule_score = schedule.copy()

    # replace schedule with actual matchup scores
    for i in range(0, schedule.shape[0]):
        for j in range(1, schedule.shape[1]):
            for k in range(0, len(score)):
                if score.loc[k, "Team"] == schedule.iloc[i, j]:
                    schedule_score.iloc[i, j] = score.iloc[k, 2]

    custom_sched_score = schedule_score.iloc[:, :(last_week+1)]

    return custom_sched_score

In [22]:
def combo_score(target_team, custom_sched_score, last_week):

    target_team_name = custom_sched_score.loc[target_team, "Team"]
    col_label = target_team_name + "_" + str(last_week)

    target_matchups = custom_sched_score.iloc[target_team, :]

    combos_calc = custom_sched_score.copy()

    sum_combos_score = []

    for i in range(0, custom_sched_score.shape[0]):
        for j in range(1, custom_sched_score.shape[1]):
            if target_matchups[j] > custom_sched_score.iloc[i, j]:
                combos_calc.iloc[i, j] = 0
            else:
                combos_calc.iloc[i, j] = custom_sched_score.iloc[i, j] - target_matchups[j]

        sum_combos_score.append(combos_calc.iloc[i, 1:].sum())

    sum_combos_score_df = pd.DataFrame({col_label: sum_combos_score})

    targ_team_combo = pd.concat([custom_sched_score.loc[:, "Team"], sum_combos_score_df, custom_sched_score.iloc[:, 1:]], axis=1)

    targ_team_combo_sorted = targ_team_combo.sort_values(by=[col_label], ascending=False)
    
    return targ_team_combo_sorted, col_label

### Inputs

In [23]:
bye_adjustor = -2.5
schedule = load_schedule()
def_score = calc_defense_strength(bye_adjustor)
off_score = calc_offense_strength(bye_adjustor)

target_team_list = range(0, 32)
last_week_list = [17]

### Make Excel File with Defense Matchup Strengths

In [8]:
# make writer object for Excel file
writer = pd.ExcelWriter('./combos/qb_combos.xlsx', engine='xlsxwriter')

# iterate through every input permutation
for i in range(0, len(last_week_list)):
    
    for j in range(0, len(target_team_list)):
        
        
        custom_sched_score = matchup_scores_by_week(schedule, def_score, last_week_list[i])
        targ_team_combo_sorted, col_label = combo_score(target_team_list[j], custom_sched_score, last_week_list[i])
        
        targ_team_combo_sorted.to_excel(writer, sheet_name=col_label)
        
        print col_label

writer.save()

ARI_15
ATL_15
BAL_15
BUF_15
CAR_15
CHI_15
CIN_15
CLE_15
DAL_15
DEN_15
DET_15
GB_15
HOU_15
IND_15
JAC_15
KC_15
LAC_15
LAR_15
MIA_15
MIN_15
NE_15
NO_15
NYG_15
NYJ_15
OAK_15
PHI_15
PIT_15
SEA_15
SF_15
TB_15
TEN_15
WAS_15
ARI_16
ATL_16
BAL_16
BUF_16
CAR_16
CHI_16
CIN_16
CLE_16
DAL_16
DEN_16
DET_16
GB_16
HOU_16
IND_16
JAC_16
KC_16
LAC_16
LAR_16
MIA_16
MIN_16
NE_16
NO_16
NYG_16
NYJ_16
OAK_16
PHI_16
PIT_16
SEA_16
SF_16
TB_16
TEN_16
WAS_16
ARI_17
ATL_17
BAL_17
BUF_17
CAR_17
CHI_17
CIN_17
CLE_17
DAL_17
DEN_17
DET_17
GB_17
HOU_17
IND_17
JAC_17
KC_17
LAC_17
LAR_17
MIA_17
MIN_17
NE_17
NO_17
NYG_17
NYJ_17
OAK_17
PHI_17
PIT_17
SEA_17
SF_17
TB_17
TEN_17
WAS_17


### Make Excel File with Offense Matchup Strengths

In [24]:
# make writer object for Excel file
writer = pd.ExcelWriter('./combos/def_combos.xlsx', engine='xlsxwriter')

# iterate through every input permutation
for i in range(0, len(last_week_list)):
    
    for j in range(0, len(target_team_list)):
        
        
        custom_sched_score = matchup_scores_by_week(schedule, off_score, last_week_list[i])
        targ_team_combo_sorted, col_label = combo_score(target_team_list[j], custom_sched_score, last_week_list[i])
        
        targ_team_combo_sorted.to_excel(writer, sheet_name=col_label)
        
        print col_label

writer.save()

ARI_15
ATL_15
BAL_15
BUF_15
CAR_15
CHI_15
CIN_15
CLE_15
DAL_15
DEN_15
DET_15
GB_15
HOU_15
IND_15
JAC_15
KC_15
LAC_15
LAR_15
MIA_15
MIN_15
NE_15
NO_15
NYG_15
NYJ_15
OAK_15
PHI_15
PIT_15
SEA_15
SF_15
TB_15
TEN_15
WAS_15
ARI_16
ATL_16
BAL_16
BUF_16
CAR_16
CHI_16
CIN_16
CLE_16
DAL_16
DEN_16
DET_16
GB_16
HOU_16
IND_16
JAC_16
KC_16
LAC_16
LAR_16
MIA_16
MIN_16
NE_16
NO_16
NYG_16
NYJ_16
OAK_16
PHI_16
PIT_16
SEA_16
SF_16
TB_16
TEN_16
WAS_16
ARI_17
ATL_17
BAL_17
BUF_17
CAR_17
CHI_17
CIN_17
CLE_17
DAL_17
DEN_17
DET_17
GB_17
HOU_17
IND_17
JAC_17
KC_17
LAC_17
LAR_17
MIA_17
MIN_17
NE_17
NO_17
NYG_17
NYJ_17
OAK_17
PHI_17
PIT_17
SEA_17
SF_17
TB_17
TEN_17
WAS_17
